In [ ]:
import os
import sys
import time
import logging
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import random
import itertools
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from itertools import permutations
import numpy as np
import itertools
import math

ModuleNotFoundError: No module named 'bool'

In [9]:
def make_pref(n,m):
    row = torch.linspace(1/m, 1, m)  # 1行分の値を生成
    pref = torch.stack([row[torch.randperm(m)] for _ in range(n)])
    return pref

In [3]:
def serial_dictatorship(n, m, preferences, order):
    # 各順列での配分行列
    allocation = torch.zeros((n, m))
    available_goods = torch.ones(m)  # 各財が未配分
    
    for i in order:
        # 参加者 i の選好を取得
        pref = preferences[i]
        # 選好の高い順に財を割り当てる
        for good in torch.argsort(pref, descending=True):
            if available_goods[good] > 0:
                allocation[i][good] = 1
                available_goods[good] = 0
                break  # 一つの財を割り当てたら終了
    
    return allocation

def random_serial_dictatorship(n, m, preferences):
    # 全順列の平均配分行列
    total_allocation = torch.zeros((n, m))
    
    # 全順列を生成
    for order in itertools.permutations(range(n)):
        sd_allocation = serial_dictatorship(n, m, preferences, order)
        total_allocation += sd_allocation
    
    # 平均を取る
    allocation = total_allocation / torch.tensor(math.factorial(n), dtype=torch.float)
    
    return allocation

In [5]:
def probabilistic_serial(n, m, preferences, allocation_step=0.001):
    # 各参加者の割り当て配分行列
    allocation = torch.zeros((n, m))
    # 各財の残り割合（全員が分配を完了するまで1.0）
    available_goods = torch.ones(m)
    
    # 分配プロセス: 各参加者が0から徐々に割り当てを行う
    while available_goods.sum() > 0:
        # 各参加者が少しずつ配分
        for i in range(n):
            # 参加者iの選好に基づき、可能な限り財を取得
            for good in torch.argsort(preferences[i], descending=True):
                if available_goods[good] > 0:
                    # 配分率（allocation_stepずつ割り当て）
                    allocation_rate = min(available_goods[good], allocation_step)
                    allocation[i][good] += allocation_rate
                    available_goods[good] -= allocation_rate
                    # 財が尽きたら次の財に移る
                    if available_goods[good] <= 0:
                        available_goods[good] = 0  # 残りが0未満にならないように補正
                    break  # 次の参加者に移る
    return allocation

In [7]:
n = 4
m = 4
preferences = torch.tensor([[1, 0.75, 0.5, 0.25],[1, 0.75, 0.5, 0.25], [1, 0.75, 0.25, 0.5], [0.75, 1, 0.25, 0.5]])

print("Preferences:")
print(preferences)

allocation = random_serial_dictatorship(n, m, preferences)
print("Allocation(RSD):")
print(allocation)

allocation = probabilistic_serial(n, m, preferences)
print("Allocation(PS):")
print(allocation)

Preferences:
tensor([[1.0000, 0.7500, 0.5000, 0.2500],
        [1.0000, 0.7500, 0.5000, 0.2500],
        [1.0000, 0.7500, 0.2500, 0.5000],
        [0.7500, 1.0000, 0.2500, 0.5000]])
Allocation(RSD):
tensor([[0.3333, 0.1667, 0.4167, 0.0833],
        [0.3333, 0.1667, 0.4167, 0.0833],
        [0.3333, 0.1667, 0.0833, 0.4167],
        [0.0000, 0.5000, 0.0833, 0.4167]])
Allocation(PS):
tensor([[3.3400e-01, 1.6700e-01, 5.0000e-01, 0.0000e+00],
        [3.3301e-01, 1.6601e-01, 5.0000e-01, 0.0000e+00],
        [3.3300e-01, 1.6700e-01, 0.0000e+00, 5.0001e-01],
        [0.0000e+00, 5.0000e-01, 9.3246e-06, 5.0000e-01]])


In [8]:
n = 4
m = 4
preferences = torch.tensor([[1, 0.75, 0.5, 0.25],[1, 0.75, 0.5, 0.25], [0.75, 1, 0.5, 0.25], [0.75, 1, 0.25, 0.5]])

print("Preferences:")
print(preferences)

allocation = random_serial_dictatorship(n, m, preferences)
print("Allocation(RSD):")
print(allocation)

allocation = probabilistic_serial(n, m, preferences)
print("Allocation(PS):")
print(allocation)

Preferences:
tensor([[1.0000, 0.7500, 0.5000, 0.2500],
        [1.0000, 0.7500, 0.5000, 0.2500],
        [0.7500, 1.0000, 0.5000, 0.2500],
        [0.7500, 1.0000, 0.2500, 0.5000]])
Allocation(RSD):
tensor([[0.4167, 0.0833, 0.3333, 0.1667],
        [0.4167, 0.0833, 0.3333, 0.1667],
        [0.0833, 0.4167, 0.3333, 0.1667],
        [0.0833, 0.4167, 0.0000, 0.5000]])
Allocation(PS):
tensor([[5.0001e-01, 0.0000e+00, 3.3301e-01, 1.6600e-01],
        [5.0000e-01, 9.3246e-06, 3.3300e-01, 1.6700e-01],
        [0.0000e+00, 5.0000e-01, 3.3400e-01, 1.6700e-01],
        [0.0000e+00, 5.0000e-01, 0.0000e+00, 5.0001e-01]])


In [20]:
def make_pref(n,m):
    row = torch.linspace(1/m, 1, m)  # 1行分の値を生成
    pref = torch.stack([row[torch.randperm(m)] for _ in range(n)])
    return pref

def generate_random_doubly_stochastic_matrix(n, m):
    # ランダムな行列を生成
    matrix = torch.rand(n, m)
    
    # 行和が1になるように正規化
    row_sums = matrix.sum(dim=1, keepdim=True)
    matrix = matrix / row_sums
    
    # 列和が1になるように正規化
    col_sums = matrix.sum(dim=0, keepdim=True)
    matrix = matrix / col_sums
    
    return matrix


preferences = make_pref(4,4)
print(preferences)
# 例としてn=4, m=4のランダムな二重確率行列を生成
random_doubly_stochastic_matrix = generate_random_doubly_stochastic_matrix(4, 4)
print(random_doubly_stochastic_matrix)

tensor([[1.0000, 0.7500, 0.5000, 0.2500],
        [1.0000, 0.7500, 0.5000, 0.2500],
        [0.7500, 1.0000, 0.2500, 0.5000],
        [0.2500, 0.7500, 0.5000, 1.0000]])
tensor([[0.0586, 0.3948, 0.2248, 0.1714],
        [0.5842, 0.1339, 0.2008, 0.3363],
        [0.2291, 0.4171, 0.1775, 0.0111],
        [0.1281, 0.0542, 0.3968, 0.4812]])


In [ ]:
from collections import defaultdict
import torch

def top_trading_cycles(preferences, initial_owners):
    """
    preferences: torch.Tensor (n x m の行列, 各行はエージェントの各財への選好（値が大きいほど好ましい）)
    initial_owners: torch.Tensor (n x m の行列, 各行はエージェントの所有アイテムを示すone-hotベクトル)
    """
    n, m = preferences.shape
    agents = list(range(n))  # エージェントを0,1,2,...,n-1として扱う
    allocation = torch.zeros_like(initial_owners)  # 最終的なマッチング結果（one-hot行列）
    remaining_agents = set(agents)  # 交換が完了していないエージェント
    
    # 初期所有者のマッピング（財のインデックス -> エージェント）
    owner_of = {torch.argmax(initial_owners[i]).item(): i for i in range(n)}
    
    while remaining_agents:
        # 1. 各エージェントが最も好むアイテムを指す（選好が最大の財のインデックスを選択）
        pointing = {agent: torch.argmax(preferences[agent]).item() for agent in remaining_agents}
        
        visited = set()
        cycle_found = False
        
        for agent in list(remaining_agents):
            if agent in visited:
                continue
            cycle = []
            current = agent
            
            while current not in visited:
                visited.add(current)
                cycle.append(current)
                if pointing[current] not in owner_of:
                    break  # 無効な所有者ならループを抜ける
                next_owner = owner_of[pointing[current]]
                current = next_owner
            
            # サイクル発見時、アイテム交換を確定
            if current in cycle:
                cycle_start = cycle.index(current)
                cycle = cycle[cycle_start:]
                
                for i in range(len(cycle)):
                    agent_id = cycle[i]
                    item_id = pointing[agent_id]
                    allocation[agent_id] = torch.zeros(m)  # 以前の所有をクリア
                    allocation[agent_id, item_id] = 1  # 新しい所有をセット
                    del owner_of[item_id]  # アイテムを市場から除去
                
                remaining_agents -= set(cycle)  # 交換完了エージェントを削除
                cycle_found = True
                break  # 1つのサイクルが処理されたら次のループへ
        
        # どのサイクルも見つからない場合、安全のためブレーク
        if not cycle_found:
            break
    
    return allocation

# サンプルデータ
def make_pref(n, m):
    """
    各エージェントのランダムな希望リスト（値が大きいほど好ましい）を生成
    """
    row = torch.linspace(1/m, 1, m)  # 1行分の値を生成
    pref = torch.stack([row[torch.randperm(m)] for _ in range(n)])
    return pref

n, m = 4, 4
preferences = make_pref(n, m)
initial_owners = torch.eye(n, m)  # 各エージェントが一意のアイテムを所有しているone-hot表現

# アルゴリズム実行
final_allocation = top_trading_cycles(preferences, initial_owners)
print(final_allocation) 


KeyError: 0

In [37]:
from collections import defaultdict
import torch

def top_trading_cycles(n, m, preferences, initial_owners):
    """
    preferences: torch.Tensor (n x m の行列, 各行はエージェントの各財への選好（値が大きいほど好ましい）)
    initial_owners: torch.Tensor (n x m の行列, 各行はエージェントの所有アイテムを示すone-hotベクトル)
    """

    agents = list(range(n))  # エージェントを0,1,2,...,n-1として扱う
    allocation = torch.zeros_like(initial_owners)  # 最終的なマッチング結果（one-hot行列）
    remaining_agents = set(agents)  # 交換が完了していないエージェント
    
    # 初期所有者のマッピング（財のインデックス -> エージェント）
    owner_of = {torch.argmax(initial_owners[i]).item(): i for i in range(n)}
    
    while remaining_agents:
        # 1. 各エージェントが最も好むアイテムを指す（選好が最大の財のインデックスを選択）
        pointing = {agent: torch.argmax(preferences[agent]).item() for agent in remaining_agents}
        
        visited = set()
        cycle_found = False
        
        for agent in list(remaining_agents):
            if agent in visited:
                continue
            cycle = []
            current = agent
            
            while current not in visited:
                visited.add(current)
                cycle.append(current)
                if pointing[current] not in owner_of:
                    break  # 無効な所有者ならループを抜ける
                next_owner = owner_of.get(pointing[current], None)
                if next_owner is None:
                    break  # 無効な所有者ならループを抜ける
                current = next_owner
            
            # サイクル発見時、アイテム交換を確定
            if current in cycle:
                cycle_start = cycle.index(current)
                cycle = cycle[cycle_start:]
                
                for i in range(len(cycle)):
                    agent_id = cycle[i]
                    item_id = pointing[agent_id]
                    allocation[agent_id] = torch.zeros(m)  # 以前の所有をクリア
                    allocation[agent_id, item_id] = 1  # 新しい所有をセット
                
                # アイテムを市場から除去
                for agent_id in cycle:
                    item_id = pointing[agent_id]
                    if item_id in owner_of:
                        del owner_of[item_id]
                
                remaining_agents -= set(cycle)  # 交換完了エージェントを削除
                cycle_found = True
                break  # 1つのサイクルが処理されたら次のループへ
        
        # どのサイクルも見つからない場合、安全のためブレーク
        if not cycle_found:
            break
    
    return allocation

# サンプルデータ
def make_pref(n, m):
    """
    各エージェントのランダムな希望リスト（値が大きいほど好ましい）を生成
    """
    row = torch.linspace(1/m, 1, m)  # 1行分の値を生成
    pref = torch.stack([row[torch.randperm(m)] for _ in range(n)])
    return pref

n, m = 4, 4
preferences = make_pref(n, m)
initial_owners = torch.eye(n, m)  # 各エージェントが一意のアイテムを所有しているone-hot表現

# アルゴリズム実行
final_allocation = top_trading_cycles(n, m, preferences, initial_owners)
print(final_allocation)

KeyError: 0

In [38]:
from collections import defaultdict
import torch

def top_trading_cycles(n, m, preferences, initial_allocation):
    """
    preferences: torch.Tensor (n x m の行列, 各行はエージェントの各財への選好（値が大きいほど好ましい）)
    initial_owners: torch.Tensor (n x m の行列, 各行はエージェントの所有アイテムを示すone-hotベクトル)
    """

    agents = list(range(n))  # エージェントを0,1,2,...,n-1として扱う
    allocation = torch.zeros_like(initial_allocation)  # 最終的なマッチング結果（one-hot行列）
    remaining_agents = set(agents)  # 交換が完了していないエージェント
    
    # 初期所有者のマッピング（財のインデックス -> エージェント）
    owner_of = {torch.argmax(initial_allocation[i]).item(): i for i in range(n)}
    
    while remaining_agents:
        # 1. 各エージェントが最も好むアイテムを指す（選好が最大の財のインデックスを選択）
        pointing = {agent: torch.argmax(preferences[agent]).item() for agent in remaining_agents}
        
        visited = set()
        cycle_found = False
        
        for agent in list(remaining_agents):
            if agent in visited:
                continue
            cycle = []
            current = agent
            
            while current not in visited:
                visited.add(current)
                cycle.append(current)
                if pointing[current] not in owner_of:
                    break  # 無効な所有者ならループを抜ける
                next_owner = owner_of.get(pointing[current], None)
                if next_owner is None:
                    break  # 無効な所有者ならループを抜ける
                current = next_owner
            
            # サイクル発見時、アイテム交換を確定
            if current in cycle:
                cycle_start = cycle.index(current)
                cycle = cycle[cycle_start:]
                
                for i in range(len(cycle)):
                    agent_id = cycle[i]
                    item_id = pointing[agent_id]
                    allocation[agent_id] = torch.zeros(m)  # 以前の所有をクリア
                    allocation[agent_id, item_id] = 1  # 新しい所有をセット
                
                # アイテムを市場から除去
                for agent_id in cycle:
                    item_id = pointing[agent_id]
                    if item_id in owner_of:
                        del owner_of[item_id]
                
                remaining_agents -= set(cycle)  # 交換完了エージェントを削除
                cycle_found = True
                break  # 1つのサイクルが処理されたら次のループへ
        
        # どのサイクルも見つからない場合、安全のためブレーク
        if not cycle_found:
            break
    
    return allocation

# サンプルデータ
def make_pref(n, m):
    """
    各エージェントのランダムな希望リスト（値が大きいほど好ましい）を生成
    """
    row = torch.linspace(1/m, 1, m)  # 1行分の値を生成
    pref = torch.stack([row[torch.randperm(m)] for _ in range(n)])
    return pref

n, m = 4, 4
preferences = make_pref(n, m)
initial_allocation = torch.eye(n, m)  # 各エージェントが一意のアイテムを所有しているone-hot表現

# アルゴリズム実行
final_allocation = top_trading_cycles(n, m, preferences, initial_allocation)
print(final_allocation)

KeyError: 2